In [1]:
import json
import pandas as pd
import numpy as np

In [2]:
date = '2017-11-29'
fn = 'DB-backup/%s/cyclings_vid1.json' % date

In [4]:
data = json.load(open(fn))

In [5]:
dfs = {}
for i, item in enumerate(data):
    if item['type']=='table':
        name = item['name']
        if name not in ( 'video2seg_temp'):
            print(i,item['name'])
            dfs[name] = pd.DataFrame(item['data'])

2 loginLog
3 Rating
4 RoadSegment
5 Users
6 Video
8 VideoRoadSeg


In [6]:
oldserver_rating = pd.read_csv('DB-backup/%s/oldserver/oldserverRating.csv' % date)
oldserver_rating['uid'] = oldserver_rating['user_id_new']
oldserver_rating.drop('user_id_new', axis=1, inplace=True)
oldserver_rating['lid'] = oldserver_rating['lid'] * -1

In [7]:
oldserver_user = pd.read_csv('DB-backup/%s/oldserver/oldserverUser.csv' % date)
oldserver_user['user_id'] = oldserver_user['user_id_new']
oldserver_user.drop('user_id_new', axis=1, inplace=True)
oldserver_user.set_index('user_id', inplace=True)

In [8]:
oldserver_loginlog = pd.read_csv('DB-backup/%s/oldserver/oldserverUserLog.csv' % date)
oldserver_loginlog['user_id_new'] = oldserver_loginlog['user_id_new']
oldserver_loginlog.drop('user_id_new', axis=1, inplace=True)
oldserver_loginlog['lid'] = oldserver_loginlog['lid'] * -1

In [9]:
new_rating = dfs['Rating'].append(oldserver_rating)
new_rating.vid = new_rating.vid.astype(str)
new_rating.uid = new_rating.uid.astype(str)
new_rating.score = new_rating.score.astype(str)
new_rating.drop_duplicates(['uid','vid'],inplace=True)
new_rating.to_csv('DB-backup/%s/Rating.csv' % date, encoding='utf8')
new_rating.shape

(2298, 15)

In [10]:
new_users = dfs['Users'].copy()
new_users.user_id = new_users.user_id.astype(int)
new_users = new_users.set_index('user_id')
new_users.fillna(oldserver_user, inplace=True)
new_users = new_users.append(oldserver_user[oldserver_user.index<=0])
new_users.reset_index().to_csv('DB-backup/%s/Users.csv' % date, encoding='utf8')
new_users.email.nunique(), new_users.shape

(244, (244, 15))

In [11]:
new_login = dfs['loginLog'].append(oldserver_loginlog)
new_login.to_csv('DB-backup/%s/loginLog.csv' % date, encoding='utf8')
new_login.shape

(443, 9)

In [12]:

dfs['Video'].to_csv('DB-backup/%s/Video.csv' % date, encoding='utf8')
dfs['VideoRoadSeg'].to_csv('DB-backup/%s/VideoRoadSeg.csv' % date, encoding='utf8')
dfs['RoadSegment'].to_csv('DB-backup/%s/RoadSegment.csv' % date, encoding='utf8')